#### 성능지표 - 분류
- 정확도(Accuracy) : 전체 정답에서 맞춘 정답의 수
- 정밀도(Precision) : 모델(예측) 기준으로 모델이 True라고 한 것이 (정답)실제도 True인 것
- 재현율(Recall) : 정답(실제) 기준으로 정답(실제)이 True라고 한 것이 모델(예측)도 True인 것
- F1-Score : 정밀도와 재현율 2가지를 조합한 점수
- 오차행렬(Confusion Matrix) : 정답과 예측값의 관계 나타내는 표

[1] 모듈 로딩 및 데이터 준비

In [1]:
# 모듈로딩
from sklearn.datasets import load_digits
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score  # sklearn에서 제공하는 모델 성능 평가 관련 모듈

import pandas as pd
import numpy as np

In [2]:
# 데이터 로딩
digitBunch = load_digits(as_frame=True)

[2] 피쳐와 레이블 추출

In [3]:
# 0 ~ 9 숫자 이미지 데이터 값 : 피쳐
imgDF = digitBunch['data']

# 0 ~ 9 라벨
labelSR = digitBunch['target']
labelSR.unique()

array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9])

In [4]:
## ==> 2진분류를 위해서 7과 나머지 라벨 변경
mask = (labelSR==7) # 7 찾기
labelSR[~mask] = 0 # 7이 아닌 나머지 0으로 변경
labelSR[mask] =1 # 7인건 1로 변경

In [5]:
# labelSR2 = (labelSR==7).astype(int)
# 이렇게도 가능

In [6]:
labelSR

0       0
1       0
2       0
3       0
4       0
       ..
1792    0
1793    0
1794    0
1795    0
1796    0
Name: target, Length: 1797, dtype: int32

In [7]:
labelSR.value_counts()

target
0    1618
1     179
Name: count, dtype: int64

[3] 학습용 데이터셋 준비

In [8]:
X_train, X_test, y_train, y_test = train_test_split(imgDF,
                                                    labelSR,
                                                    stratify=labelSR,
                                                    random_state=1)

In [9]:
print('[Train]', y_train.value_counts()/y_train.shape[0], y_train.shape)
print('[Test]', y_test.value_counts()/y_test.shape[0], y_test.shape)

[Train] target
0    0.90052
1    0.09948
Name: count, dtype: float64 (1347,)
[Test] target
0    0.9
1    0.1
Name: count, dtype: float64 (450,)


In [10]:
print('[Train]', y_train.value_counts(), y_train.shape)
print('[Test]',  y_test.value_counts(), y_test.shape)

[Train] target
0    1213
1     134
Name: count, dtype: int64 (1347,)
[Test] target
0    405
1     45
Name: count, dtype: int64 (450,)


In [11]:
y_train

1158    0
657     0
1120    0
1408    0
680     0
       ..
1542    0
518     0
565     0
1750    0
909     0
Name: target, Length: 1347, dtype: int32

지금까지의 과정 [OvR 가능 - 다중분류]

[4] 클래스 정의 

In [12]:
# 사용자 정의 클래스  DummyClass : 아무것도 하지 않는 클래스, 단순 테스트용
from sklearn.base import BaseEstimator

class MyClassification(BaseEstimator):

    def fit(self,X, y ) : pass

    def predcit(self, X) : return np.zeros((len(X), 1), dtype=bool)

    def predcit1(self, X) : return np.zeros((len(X), 1), dtype='int8')

[5] 학습


In [13]:
# 인스턴스 생성
model = MyClassification()
# 학습
model.fit(X_train, y_train)

[6] 예측 및 성능 평가

In [14]:
pre_1= model.predcit1(X_train)
pre_1

array([[0],
       [0],
       [0],
       ...,
       [0],
       [0],
       [0]], dtype=int8)

In [15]:
pre_ = model.predcit(X_train)
pre_

array([[False],
       [False],
       [False],
       ...,
       [False],
       [False],
       [False]])

[6-1] 정확도=> 불균형 데이터 경우 신뢰 불가!!!!

In [16]:
accuracy_score(y_train, pre_ )

0.9005196733481812

[6-2] 오차행렬  

In [17]:
from sklearn.metrics import confusion_matrix, classification_report

In [18]:
# 정답과 예측값 전달
(tn, fp, fn, tp) = confusion_matrix(y_train, pre_).reshape(-1)
print(tn, fp, fn, tp)

1213 0 134 0


In [19]:
confusion_matrix(y_train, pre_)

array([[1213,    0],
       [ 134,    0]], dtype=int64)

In [20]:
y_test1 = [0,0,0,1,1,  1,0,0,0,1,  1,1,1,0,0,  0,0,0,0,0]
y_pre = [1,1,1,1,1,  1,0,1,0,1,  1,1,1,0,0,  0,0,0,0,0]

In [21]:
# 정답과 예측값 전달
(tn, fp, fn, tp) = confusion_matrix(y_test1, y_pre).reshape(-1)
print(tn, fp, fn, tp)

9 4 0 7


In [22]:
confusion_matrix(y_test1, y_pre)

array([[9, 4],
       [0, 7]], dtype=int64)

- 정밀도 (Precision) : 모델(예측) 중심!!

In [23]:
from sklearn.metrics import precision_score
precision_score(y_train, pre_, zero_division=0)

0.0

In [24]:
precision_score(y_test1, y_pre, zero_division=0)

0.6363636363636364

- 재현율(Recall) : 정답(실제) 중심!!

In [25]:
from sklearn.metrics import recall_score
recall_score(y_train, pre_)

0.0

In [26]:
recall_score(y_test1, y_pre)

1.0